In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

def plot_isogeny_offsets(
    data,
    execution_times,
    output_file=None,
    use_latex=True
):
    if use_latex:
        plt.rc("text", usetex=True)

    # Define palette
    palette = sns.color_palette("Set1", 12).as_hex()[:12]
    palette1 = sns.color_palette("Set2", 3).as_hex()[:3]



    data.loc[:,'expected_degree_str'] = pd.Categorical(data.loc[:,'expected_degree_str'])
    data = data.sort_values('expected_degree_str')
    # Scatter plot
    ax = sns.scatterplot(
        data=data,
        x="ext_offset",
        y="expected_degree_str",
        hue="expected_degree_str",
        palette=palette,
        # bins=50
    )

    ax.set_xlabel("Ext. offset")
    ax.set_ylabel("Distance")

    intervals = data.interval.unique()
    bgmap = {
        execution_times.iloc[i]['degree']: palette[j] for j,i in enumerate(intervals)
    }
    # Background spans
    for i in intervals:
        xmin = execution_times.iloc[i]['setup_start']
        xmax = execution_times.iloc[i]['isogeny_end']
        l = execution_times.iloc[i]['degree']
        plt.axvspan(xmin=xmin, xmax=xmax, alpha=0.1, color=bgmap[l])

    for i in intervals:
        plt.vlines(execution_times.iloc[i]['isogeny_start'], ymin=-1, ymax=12)


    # Proxy patches for background legend
    proxy_patches = [
        Patch(facecolor=color, alpha=0.8, label=f"{int(deg)}")
        for deg, color in sorted(bgmap.items(), key=lambda x: abs(x[0]))
    ]

    #Legend for background
    leg2 = ax.legend(
        handles=proxy_patches,
        title="Background (degree)",
        loc="upper left",
        bbox_to_anchor=(1, 0.3)  # adjust to control stacking
    )

    #Keep both legends
    # ax.add_artist(leg1)

    # Output
    if output_file:
        plt.savefig(output_file, bbox_inches="tight")
    else:
        plt.show()
